In [ ]:
from rasterio.plot import show
import os
import rasterio as rio
import cv2
import math
import numpy as np

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 100
plt.rcParams.update({'font.size': 6})

In [ ]:
def getSingleTifFromPath():
    user_input = input('Enter the folder path to a sample GeoTIFF image:\n')
    test_path = f'{user_input}'
    if os.path.exists(test_path): 
        return rio.open(os.path.join(test_path))
    else:
        print("The supplied folder path does not exist, please try again!")
        return None

In [ ]:
test_image_full = getSingleTifFromPath()
print(test_image_full)

In [ ]:
show(test_image_full)

In [ ]:
test_image_full.meta

In [ ]:
test_image_full.close()

In [ ]:
def scale_min_max_contours(m, r_min, r_max, t_min, t_max):
#     print(m[0], m[1])
    x_scale = (m[0] - t_min)/(t_max - t_min) * (r_max - r_min) + r_min
    y_scale = (m[1] - t_min)/(t_max - t_min) * (r_max - r_min) + r_min
    return (int(x_scale), int(y_scale))

In [ ]:
def get_contour_scale(dim, points):
    t_min = round(np.array(points).min(),0)
    t_max = round(np.array(points).max(),0)
    return 0, dim-1, int(t_min), int(t_max)

In [ ]:
def build_flat_contour_array(img):
    width, length = img.shape[0], img.shape[1]
    zero_matrix = np.zeros((width, length), int)
    x, y = np.linspace(0, width, width), np.linspace(0, length, length)
    X, Y = np.meshgrid(x, y)
    cs = plt.contour(X, Y, img)
    contour_points = []
    for i, collection in enumerate(cs.collections[1:]):
        for path in collection.get_paths():
            if path.to_polygons():
                for npoly, polypoints in enumerate(path.to_polygons()):
                    for polypoint in polypoints:
                        contour_points.append((polypoint[0], polypoint[1]))
                        
    r_min, r_max, t_min, t_max = get_contour_scale(width, contour_points)
    print(r_min, r_max, t_min, t_max)
    contour_points = np.apply_along_axis(scale_min_max_contours, 1, contour_points, r_min, r_max, t_min, t_max)
    
#     contour_points = np.apply_along_axis(scale_min_max_contours, 1, contour_points, 0, 99, 0, 100)
    for point in contour_points:
        zero_matrix[point[0]][point[1]] = 1
    plt.imshow(zero_matrix.T)
    plt.show()
#     plt.hist(zero_matrix.T)
#     plt.show()
    return zero_matrix.T.flatten()

In [ ]:
def plotRGBNIR(img_path):
    rio_img = rio.open(img_path)
    plt.subplot(221),plt.imshow(rio_img.read(1), 'Reds'),plt.title("Red Band")
    plt.subplot(222),plt.imshow(rio_img.read(2), 'Blues'),plt.title("Green Band")
    plt.subplot(223),plt.imshow(rio_img.read(3), 'Greens'),plt.title("Blue Band")
    plt.subplot(224),plt.imshow(rio_img.read(4), 'Spectral_r'),plt.title("NIR Band")
    plt.subplots_adjust(hspace=0.5)

In [ ]:
def getFolderPath():
    user_input = input('Enter the folder path to where sample TIF images are located:\n')
    test_path = f'{user_input}'
    if os.path.exists(test_path): 
        return test_path
    else:
        print("The supplied folder path does not exist, please try again!")
        return None

In [ ]:
# this folder path is expecting the "annotation subset" folder
folder_path = getAllImagesFromPath()

In [ ]:
# this is a sample of spliced HRO images representing nature and man made structures
sample_nature = '10seg730865_700-1400.tif'
sample_nature2 = '10seg985865_2100-4800.tif'
sample_nature3 = '10seg985850_2400-3100.tif'
sample_pool = '10seg985850_300-3700.tif'
sample_house = '10seg745865_4700-1000.tif'
sample_road = '10seg745880_400-1200.tif'
sample_parkinglot = '10seg745880_4600-4900.tif'
sample_solar = '10seg895850_2400-200.tif'

sample_set = [['nature1', sample_nature, '1'],
              ['nature2', sample_nature2, '1'],
              ['nature3', sample_nature3, '1'],
              ['house', sample_house, '0'],
              ['road', sample_road, '0'],
              ['parking', sample_parkinglot, '0'],
              ['solar', sample_solar, '0'],
              ['pool', sample_pool, '0']]

In [ ]:
plotRGBNIR(os.path.join(folder_path, 'cluster_0', sample_solar))

In [ ]:
plotRGBNIR(os.path.join(folder_path, 'cluster_1', sample_nature))

In [ ]:
sample_img = rio.open(os.path.join(folder_path, 'cluster_0', sample_house))
for ii in range(1, sample_img.meta['count']+1):
    print('-'*80)
    print(np.percentile(sample_img.read(ii), 5))
    print(np.median(sample_img.read(ii)))
    print(np.percentile(sample_img.read(ii), 95))

In [ ]:
show(sample_img)

In [ ]:
show(sample_img.read(4))

In [ ]:
sample_img = rio.open(os.path.join(folder_path, 'cluster_0', sample_solar))
for ii in range(1, sample_img.meta['count']+1):
    print('-'*80)
    print(np.percentile(sample_img.read(ii), 5))
    print(np.median(sample_img.read(ii)))
    print(np.percentile(sample_img.read(ii), 95))

In [ ]:
show(sample_img.read(4))

In [ ]:
show(sample_img)

In [ ]:
sample_img = rio.open(os.path.join(folder_path, 'cluster_1', sample_nature2))
for ii in range(1, sample_img.meta['count']+1):
    print('-'*80)
    print(np.percentile(sample_img.read(ii), 5))
    print(np.median(sample_img.read(ii)))
    print(np.percentile(sample_img.read(ii), 95))

In [ ]:
show(sample_img.read(4))

In [ ]:
show(sample_img)

### Generate pixelwise histogram of average pixel value by specified color band

In [ ]:
def generatePixelRowDistribution(path, cluster, tif, band):
    image = rio.open(os.path.join(path, cluster, tif))
    n = image.meta['width']
    pixel_dist = [0]*n
    for row in image.read(band):
        for cell in range(len(row)):
            pixel_dist[cell] += row[cell]
    pixel_dist = [round(val/n, 0) for val in pixel_dist]
    plt.title(f'Row-wise pixel value distribution for band {band} - {tif} - {cluster}')
    plt.plot(pixel_dist)
    plt.show()

In [ ]:
for sample in sample_set:
    generatePixelRowDistribution(folder_path, f'cluster_{sample[2]}', sample[1], 4)

### Using PCA to determine if components can be built from image contours

In [ ]:
flat_contours = []
for sample in sample_set:
    print(f'Opening image and gathering contours for {sample[0]}')
    rio_img = rio.open(os.path.join(folder_path, f'cluster_{sample[2]}', sample[1]))
    show(rio_img)
    cv_img = cv2.imread(os.path.join(folder_path, f'cluster_{sample[2]}', sample[1]), 2)
    flat_contours.append(build_flat_contour_array(cv_img))
    rio_img.close()

In [ ]:
for li in flat_contours:
    plt.hist(li, bins=10)
    plt.show()

In [ ]:
np.array(flat_contours).T.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(np.array(flat_contours).T)
print(plt.plot(pca.explained_variance_ratio_))

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
all_contours = []
tif_dir = os.path.join(folder_path, 'cluster_0')

#### Warning, this next cell takes a long time to run because it generates contours with CV2 library on potentially thousands of TIF images, depending on the folder you specify

In [ ]:
for image in [tif for tif in os.listdir(tif_dir) if tif.__contains__('.tif')]:
    cv_img = cv2.imread(os.path.join(tif_dir, image), 2)
    all_contours.append(build_flat_contour_array(cv_img))

In [ ]:
pca2 = PCA(.8)
pca_mat = pca2.fit_transform(np.array(all_contours))
print(pca2.explained_variance_ratio_)
plt.plot(pca2.explained_variance_ratio_)

In [ ]:
sum(pca2.explained_variance_ratio_[0:1400])

In [ ]:
len(pca2.explained_variance_ratio_)

In [ ]:
pca2.components_

### Everything below this point is experimental and code gaps exist, run at your own risk

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['contours'] = flat_contours
df = pd.concat([df[col].apply(pd.Series) for col in df.columns], axis=1, ignore_index=True)
col_count = df.shape[1]
df['target'] = [0,0,0,1,1,1,1,1]

In [ ]:
col_count

In [ ]:
df.shape[1]

In [ ]:
df.head(8)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km = KMeans(n_clusters=2)
contour_cols = np.linspace(0,9999, 10000)
X = df[contour_cols].values
km.fit(X)

In [ ]:
df['cluster'] = km.labels_

In [ ]:
df['diff'] = df.apply(lambda row: 1 if row['target'] == row['cluster'] else 0, axis=1)

In [ ]:
sample_set[:]

In [ ]:
file_list = [sample[1] for sample in sample_set]

In [ ]:
df['file'] = file_list

In [ ]:
df.head()

In [ ]:
df.loc[df['file']=='10seg730865_700-1400.tif']['file']

In [ ]:
rio_solar = rio.open(os.path.join(folder_path, sample_solar))
rio_forest = rio.open(os.path.join(folder_path, sample_nature))

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
show((rio_solar, 1), cmap='Greys_r', interpolation='none', ax=ax)
show((rio_solar, 1), contour=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
show((rio_forest, 1), cmap='Greys_r', interpolation='none', ax=ax)
show((rio_forest, 1), contour=True, ax=ax)
plt.show()

In [ ]:
x, y = np.linspace(0,100, 100), np.linspace(0,100, 100)
X, Y = np.meshgrid(x, y)
plt.contourf(X, Y, img, 20, cmap='RdGy')
plt.colorbar();

In [ ]:
contours = plt.contour(X, Y, img)

In [ ]:
img_cv = cv2.imread(os.path.join(folder_path, sample_solar))
imgray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, threshold = cv2.findContours(thresh, cv2.THRESH_TOZERO, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
cv2.drawContours(img_cv, im2, -1, (0,255,0), 3)
cv2.imshow('Contours', img_cv)
cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# cs = plt.contourf(rio_forest, levels=[10, 30, 50],
#     colors=['#808080', '#A0A0A0', '#C0C0C0'], extend='both')
# cs.cmap.set_over('red')
# cs.cmap.set_under('blue')
# cs.changed()

In [ ]:
# im = cv2.imread('test.jpg')
# imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
# ret,thresh = cv2.threshold(imgray,127,255,0)
# image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
forest_img = cv2.imread(os.path.join(folder_path, sample_nature), 2)
edges = cv2.Canny(forest_img,25,255)

plt.subplot(121),plt.imshow(forest_img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
build_flat_contour_array(forest_img)

In [ ]:
# >>> cs = plt.contour(x,y,m, [9.5])
# >>> cs.collections[0].get_paths()
# p = cs.collections[0].get_paths()[0]
# v = p.vertices
# x = v[:,0]
# y = v[:,1]

# cs = plt.contour(X, Y, forest_img)
# p = cs.collections.get_paths()

In [ ]:
parking_img = cv2.imread(os.path.join(folder_path, sample_parkinglot), 2)
edges = cv2.Canny(parking_img,25,255)

plt.subplot(121),plt.imshow(parking_img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
# cs = plt.contour(X, Y, parking_img)

In [ ]:
build_flat_contour_array(parking_img)

In [ ]:
house_img = cv2.imread(os.path.join(folder_path, sample_house), 2)
edges = cv2.Canny(house_img,25,255)

plt.subplot(121),plt.imshow(house_img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
# cs = plt.contour(X, Y, house_img)

In [ ]:
build_flat_contour_array(house_img)

In [ ]:
road_img = cv2.imread(os.path.join(folder_path, sample_road), 2)
edges = cv2.Canny(road_img,25,200)

plt.subplot(121),plt.imshow(road_img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
# cs = plt.contour(X, Y, road_img)

In [ ]:
build_flat_contour_array(road_img)

In [ ]:
solar_img = cv2.imread(os.path.join(folder_path, sample_solar), 2)
edges = cv2.Canny(solar_img,100,255)

plt.subplot(121),plt.imshow(solar_img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
# cs = plt.contour(X, Y, solar_img)

In [ ]:
build_flat_contour_array(solar_img)

In [ ]:
ret,thresh1 = cv2.threshold(img,50,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,50,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,50,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,50,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,50,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

In [ ]:
thresh3

In [ ]:
ret,th1 = cv2.threshold(img,50,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

In [ ]:
# global thresholding
ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# for sample in sample_set:
#     im = rio.open(os.path.join(folder_path, sample[1]))
#     show_hist(im, bins=50, lw=0.0, stacked=False, alpha=0.3,
#          histtype='stepfilled', title=f'{sample[0]} Histogram')

In [ ]:
image_path = 'D://Raster//HRO//parsed_ortho_files//bay_area_ca_0.3m_tiles//'

In [ ]:
image_files = os.listdir(image_path)

In [ ]:
test_forest = '10seg745835_0-1100.tif'
test_house = '10seg925850_1100-100.tif'

In [ ]:
test_forest_image = rio.open(os.path.join(image_path, test_forest))
test_house_image = rio.open(os.path.join(image_path, test_house))

In [ ]:
# plt.imshow(test_image.read(2))
# plt.show()

In [ ]:
show(test_forest_image)
# show((src, 2), cmap='viridis')
# show(src.read(2), transform=src.transform, cmap='viridis')

In [ ]:
show((test_forest_image,1), cmap='viridis')
# show(test_forest_image.read(2), transform=test_forest_image.transform, cmap='viridis')

In [ ]:
show(test_house_image)

In [ ]:
show((test_house_image,1), cmap='viridis')

In [ ]:
show((test_house_image,2), cmap='viridis')

In [ ]:
show((test_house_image,3), cmap='viridis')

In [ ]:
show((test_house_image,4), cmap='viridis')

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
show((test_house_image, 1), cmap='Greys_r', interpolation='none', ax=ax)
show((test_house_image, 1), contour=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
show((test_forest_image, 1), cmap='Greys_r', interpolation='none', ax=ax)
show((test_forest_image, 1), contour=True, ax=ax)
plt.show()

In [ ]:
# >>> fig, ax = pyplot.subplots(1, figsize=(12, 12))
# >>> show((src, 1), cmap='Greys_r', interpolation='none', ax=ax)
# <matplotlib.axes._subplots.AxesSubplot object at 0x...>
# >>> show((src, 1), contour=True, ax=ax)
# <matplotlib.axes._subplots.AxesSubplot object at 0x...>
# >>> pyplot.show()

In [ ]:
from rasterio.plot import show_hist

In [ ]:
show_hist(test_house_image, bins=50, lw=0.0, stacked=False, alpha=0.3,
         histtype='stepfilled', title='House Histogram')

In [ ]:
show_hist(test_forest_image, bins=50, lw=0.0, stacked=False, alpha=0.3,
         histtype='stepfilled', title='Forest Histogram')

In [ ]:
# >>> from rasterio.plot import show_hist
# >>> show_hist(
# ...     src, bins=50, lw=0.0, stacked=False, alpha=0.3,
# ...     histtype='stepfilled', title="Histogram")

In [ ]:
test_image.close()

In [ ]:
from skimage import measure
import numpy as np

In [ ]:
test_house_image.read(1)/255

In [ ]:
x, y = np.ogrid[-np.pi:np.pi:100j, -np.pi:np.pi:100j]
r = np.sin(np.exp((np.sin(x)**3 + np.cos(y)**2)))

In [ ]:
contours = measure.find_contours(r, .8)

In [ ]:
r

In [ ]:
house_contours = measure.find_contours(test_house_image.read(1)/255, .2)

In [ ]:
# house_contours

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_house_image.read(1), cmap=plt.cm.gray)

for contour in house_contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
forest_contours = measure.find_contours(test_forest_image.read(1)/255, .2)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_forest_image.read(1), cmap=plt.cm.gray)

for contour in forest_contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
nature_list = ['10seg925850_1000-4600.tif', '10seg745835_0-1100.tif', '10seg925850_1100-1300.tif', '10seg925850_1100-2000.tif', '10seg925850_1100-3900.tif', '10seg955835_4000-100.tif']
house_list = ['10seg925850_1100-300.tif', '10seg925850_1100-100.tif', '10seg925850_1200-300.tif', '10seg925850_1200-400.tif', '10seg955835_3900-3900.tif','10seg955835_3900-3800.tif']

In [ ]:
def get_contours(in_arr, const=.2):
    return measure.find_contours(in_arr/255, const)

In [ ]:
def plot_img_contours(img, band, contours):
    fig, ax = plt.subplots()
    ax.imshow(img.read(band), cmap=plt.cm.gray)

    for contour in contours:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

    ax.axis('image')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()